# Paper Survey - A disease inference method based on symptom extraction and bidirectional Long Short Term Memory networks
There are three steps involved in the project:
1. **Symptom extraction**: The primary task is the preprocessing of electronic medical texts and the identification of symptom entities. In this part, after filtering out the no-symptom entities from the medical texts according to
the structural characteristics of MIMIC-III, we use the existing natural
language processing tool MetaMap to identify symptom entities
which are extracted from full clinical texts.
2. **Symptom representation**: Then the vector representation
of symptoms is obtained based on two representations, in which the TF-IDF obtains the strength of the association of each symptom with all the diseases and uses this as an element of the symptom vector. At the same time, the preprocessed text is used to train Word2Vec to obtain the word vector which is utilized to generate the symptom vector.
3. **BiLSTMs**: The third part is a multi-label classifier. In this part,
we use the symptom sequences to train the multi-label classification
model BiLSTMs with two symptom vector representations. The models
with different symptom representations are trained separately. Finally,
we take the weighted sum of the outputs of BiLSTMs with different
symptom representations for predicting candidate diseases.

## Install, Import and Configure required libraries

In [1]:
import nltk
from nltk.tokenize import *
import pandas as pd
import re
import json
import requests
from typing import List, Tuple
import time
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import numpy as np
import os
import re
import numpy_indexed as npi

# STEP 0: Import the MIMIC-III data

In this step we will import the NOTEEVENTS.csv.gz and filter on only "discharge summary"

In [2]:
mimic_data = pd.read_csv("NOTEEVENTS.csv",  dtype = str)

In [3]:
mimic_data.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [4]:
mimic_data.shape

(2083180, 11)

#### We will filter on Discharge summary

In [5]:
mimic_discharge_summary = mimic_data[(mimic_data.CATEGORY == "Discharge summary")]

In [6]:
mimic_discharge_summary.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [7]:
mimic_discharge_summary.shape

(59652, 11)

# **STEP 1: Symptom Extraction**

#### CHARTTIME, STORETIME, CGID and ISERROR column has all null, drop these columns

In [8]:
mimic_discharge_summary = mimic_discharge_summary.dropna(axis = 1, how = "all")

In [10]:
mimic_discharge_summary[mimic_discharge_summary.HADM_ID == "117162"]

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT
3151,2832,18689,117162,2115-06-01,Discharge summary,Report,Admission Date: [**2115-5-30**] Dischar...
50908,55416,18689,117162,2115-06-02,Discharge summary,Addendum,"Name: [**Known lastname 2132**], [**Known fir..."


#### We have few rows for Addendum to the original discharge summary, we have to merge these Addendum to the original discharge summary

In [11]:
def concat_text(group):
    main_rows = group["DESCRIPTION"] == "Report"
    if main_rows.any():
        main_text = group.loc[group["DESCRIPTION"] == "Report", "TEXT"].values[0]
        append_texts = group.loc[group["DESCRIPTION"] == "Addendum", "TEXT"].tolist()
        concatenated_text = main_text + " " + " ".join(append_texts)
        group.loc[group["DESCRIPTION"] == "Report", "TEXT"] = concatenated_text
    return group

In [12]:
grouped_mimic_discharge_summary = mimic_discharge_summary.groupby("HADM_ID", group_keys=False).apply(concat_text)

In [13]:
grouped_mimic_discharge_summary.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT
0,174,22532,167853,2151-08-04,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...


In [14]:
grouped_mimic_discharge_summary.reset_index(drop=True, inplace=True)

In [15]:
discharge_summary = grouped_mimic_discharge_summary[grouped_mimic_discharge_summary["DESCRIPTION"] != "Addendum"]

#### According to authors in paper, we will drop rows which does not have Social History, Medications on Admission, and Discharge Diagnosis.

In [23]:
strings_to_search = ['Social History', 'Medications on Admission', 'Discharge Diagnosis']
discharge_filter = discharge_summary["TEXT"].apply(lambda text: \
                                                   all([pd.Series(text, dtype='str').str.contains(s, case=False).any() \
                                                        for s in strings_to_search]))
final_discharge_summary = discharge_summary[discharge_filter]
final_discharge_summary.reset_index(drop=True, inplace=True)
final_discharge_summary.shape

(37344, 7)

In [24]:
final_discharge_summary = final_discharge_summary.drop_duplicates(subset="HADM_ID", keep = "last")
final_discharge_summary.shape

(36948, 7)

In [25]:
final_discharge_summary.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT
0,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
1,177,13702,196489,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
2,178,26880,135453,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...
3,179,53181,170490,2172-03-08,Discharge summary,Report,Admission Date: [**2172-3-5**] D...
4,180,20646,134727,2112-12-10,Discharge summary,Report,Admission Date: [**2112-12-8**] ...


#### Preporcess TEXT

In [26]:
hadm_symptom_mapping_list = []

In [27]:
def read_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                for line in file:
                    content = line.split("|")
                    hadm_id = filename.split(".")[0]
                    if hadm_id == content[0].split(".")[0]:
                        hadm_symptom_mapping_list.append((hadm_id, (content[3])))
                            
folder_path = 'discharge_summary_files/full_input'
read_files_in_folder(folder_path)

In [28]:
patient_mmap = pd.DataFrame(hadm_symptom_mapping_list, columns=["HADM_ID", "UMLS_CONCEPT"])
patient_mmap.head()

,HADM_ID,UMLS_CONCEPT
0,134788,Dyspnea
1,134788,Pulmonary Emphysema
2,134788,Chronic Kidney Diseases
3,134788,Myalgia
4,134788,Hemoptysis


In [29]:
patient_mmap.shape

(1766446, 2)

In [30]:
patient_umls = pd.merge(final_discharge_summary, patient_mmap, on="HADM_ID", how='inner')

In [31]:
patient_umls.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT,UMLS_CONCEPT
0,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,Pain
1,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,Hiatal Hernia
2,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,Wheezing
3,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,Erythema
4,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,Tracheomalacia


##### Further preprocessing
Drop CATEGORY, DESCRIPTION column as they have only one unique value
Category -> Discharge summary and Description -> Report
Drop text column as we no longer need it we have extracted the UMLS concepts from MMAP

In [32]:
patient_umls = patient_umls.drop(columns = ["CATEGORY", "DESCRIPTION", "TEXT"])
patient_umls.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,UMLS_CONCEPT
0,176,13702,167118,2119-05-25,Pain
1,176,13702,167118,2119-05-25,Hiatal Hernia
2,176,13702,167118,2119-05-25,Wheezing
3,176,13702,167118,2119-05-25,Erythema
4,176,13702,167118,2119-05-25,Tracheomalacia


In [35]:
symptom_counts = patient_umls['UMLS_CONCEPT'].value_counts()
filtered_symptoms = symptom_counts[symptom_counts >= 10].index
filtered_symptoms = set(filtered_symptoms) - {'symptom', 'illness', 'Communicable Diseases'}

filtered_patient_umls = patient_umls[patient_umls['UMLS_CONCEPT'].isin(filtered_symptoms)]

# Group by HADM_ID and aggregate UMLS_CONCEPTs as lists
grouped_patient_umls = filtered_patient_umls.groupby('HADM_ID')\
                                            .agg({'UMLS_CONCEPT': list, \
                                                  'ROW_ID': 'first', \
                                                  'SUBJECT_ID': 'first', \
                                                  'CHARTDATE': 'first'})

# Filter discharge summaries with symptoms within the range of 2 to 50
grouped_patient_umls['SYMPTOM_COUNT'] = grouped_patient_umls['UMLS_CONCEPT'].apply(len)
filtered_grouped_patient_umls = grouped_patient_umls[grouped_patient_umls['SYMPTOM_COUNT'] >= 2]

# Retain the first 50 symptoms for summaries with more than 50 symptoms
def retain_first_50(symptoms):
    return symptoms[:50]

filtered_grouped_patient_umls['UMLS_CONCEPT'] = filtered_grouped_patient_umls['UMLS_CONCEPT'].apply(retain_first_50)

# Reset the index after filtering
filtered_grouped_patient_umls.reset_index(inplace=True)
filtered_grouped_patient_umls.head()

,HADM_ID,UMLS_CONCEPT,ROW_ID,SUBJECT_ID,CHARTDATE,SYMPTOM_COUNT
0,100001,"[Chronic Kidney Diseases, Diabetic Ketoacidosi...",42102,58526,2117-09-17,44
1,100003,"[Hepatitis C, Pain, Liver Cirrhosis, Esophagea...",19215,54610,2150-04-21,51
2,100007,"[Pain, Pneumonia, Hematuria, Abdominal Pain, D...",50238,23018,2145-04-07,46
3,100009,"[Pain, Coronary Artery Disease, Tachycardia, V...",21119,533,2162-05-21,37
4,100010,"[Pain, Hematuria, Renal Cell Carcinoma, Urinar...",40054,55853,2109-12-14,32


In [36]:
filtered_grouped_patient_umls.shape

(36948, 6)

#### Creating symptom vector representations:
* Represent the extracted symptoms using the TF-IDF model and Word2Vec. You may use libraries like Scikit-learn for TF-IDF and Gensim for Word2Vec.
* Train the Word2Vec skip-gram model on the raw discharge summaries with a window size of 5 and a vector dimension of 128.

In [38]:
corpus = filtered_grouped_patient_umls['UMLS_CONCEPT'].apply(lambda x: ' '.join(x)).tolist()
corpus_raw = ' '.join(corpus)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

idf = tfidf_vectorizer.idf_
tfidf_weights = dict(zip(tfidf_vectorizer.get_feature_names_out(), idf))

# b. Train the Word2Vec
raw_sentences = simple_preprocess(corpus_raw)
raw_w2v_model = Word2Vec(window=5, vector_size=100, min_count=3)
raw_w2v_model.build_vocab([raw_sentences], progress_per=10000)
raw_w2v_model.train([raw_sentences], total_examples=raw_w2v_model.corpus_count, epochs=10)

(100000, 31815530)

We will now define a function that uses the word2vec model and the TFIDF model to obtain our vector

In [39]:
def get_output_vector(phrase):
    words = nltk.word_tokenize(phrase)
    #word2vec_vectors = []
    #tfidf_vectors = []
    vectors = []
    has_value=False
    for word in words:
        if any(w.isalpha() for w in word):
            word = word.lower()
            try:
                word2vec_vector = raw_w2v_model.wv.get_vector(word)
                tfidf_vector = tfidf_weights[word]
                op_vector = word2vec_vector * tfidf_vector
                #word2vec_vectors.append(word2vec_vector)
                #tfidf_vectors.append(tfidf_vector)
                vectors.append(op_vector)
            except KeyError:
                continue
            else:
                has_value=True
    # return np.mean(np.array(word2vec_vectors),axis=0),np.mean(np.array(tfidf_vectors),axis=0)
    if has_value:
        return torch.mean(torch.tensor(vectors),axis=0)
    else:
        return None

get_output_vector('Acute appendicitis with perforation')

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/300185627.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_2a19nf9hj1/croot/pytorch_1675190251927/work/torch/csrc/utils/tensor_new.cpp:204.)
  return torch.mean(torch.tensor(vectors),axis=0)


tensor([-0.1625,  0.6104, -0.0506,  0.2440,  0.2059, -0.7830,  0.3284,  0.9158,
        -0.4759, -0.2274, -0.1188, -0.8359, -0.0753,  0.3254, -0.0245, -0.4491,
         0.2411, -0.5197, -0.1897, -0.9636,  0.2608,  0.2770,  0.2923, -0.1891,
         0.0220,  0.0795, -0.2599, -0.2148, -0.2935,  0.0196,  0.2488,  0.0355,
         0.1956, -0.4394, -0.3878,  0.4624,  0.0748, -0.3770, -0.2339, -0.7320,
         0.0509, -0.4920, -0.2550,  0.0464,  0.4831, -0.1799, -0.3402, -0.3869,
         0.3908,  0.1591,  0.1792, -0.4478,  0.1075,  0.0081, -0.1708,  0.1481,
         0.1791,  0.0978, -0.4154,  0.3576,  0.1425, -0.1824,  0.1595, -0.1279,
        -0.4715,  0.3380,  0.2976,  0.5599, -0.7242,  0.6797, -0.2643,  0.3359,
         0.7137, -0.1571,  0.5418,  0.1156,  0.0456, -0.1833, -0.4738,  0.0966,
        -0.3462,  0.1245, -0.6028,  0.5132, -0.1917, -0.0763,  0.3530,  0.5156,
         0.4589, -0.0204,  0.6099,  0.3594, -0.0545,  0.1349,  0.7873,  0.4803,
         0.3620, -0.2243,  0.1528, -0.01

#### Preparing the dataset for multi-label classification:
* Perform disease inference tasks for the 50 most common diseases and 100 most common diseases separately. Extract the respective discharge summaries for both tasks.
* Divide the datasets into training (80%), validation (10%), and test (10%) sets randomly.

In [41]:
# load diagnoses file
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv.gz')

# Get top 50 ICD9 codes
top50_icd9 = diagnoses_icd.groupby(["ICD9_CODE"]).count().sort_values("ROW_ID", ascending = False)[: 50].index
top100_icd9 = diagnoses_icd.groupby(["ICD9_CODE"]).count().sort_values("ROW_ID", ascending = False)[: 100].index
all_icd=diagnoses_icd.groupby(["ICD9_CODE"]).count().sort_values("ROW_ID", ascending = False).index

# Get HADM_ID which has top 50 ICD9 codes
top50_hadmid = diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9)]["HADM_ID"].unique()
top100_hadmid = diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9)]["HADM_ID"].unique()

# Filter data from filtered_grouped_patient_umls with only those HADM_ID
filtered_grouped_patient_umls["HADM_ID"] = filtered_grouped_patient_umls["HADM_ID"].astype('int64')
top_50_patient_umls = filtered_grouped_patient_umls[filtered_grouped_patient_umls["HADM_ID"].isin(top50_hadmid)]
top_100_patient_umls = filtered_grouped_patient_umls[filtered_grouped_patient_umls["HADM_ID"].isin(top100_hadmid)]

top_50_patient_umls.head()

,HADM_ID,UMLS_CONCEPT,ROW_ID,SUBJECT_ID,CHARTDATE,SYMPTOM_COUNT
0,100001,"[Chronic Kidney Diseases, Diabetic Ketoacidosi...",42102,58526,2117-09-17,44
1,100003,"[Hepatitis C, Pain, Liver Cirrhosis, Esophagea...",19215,54610,2150-04-21,51
2,100007,"[Pain, Pneumonia, Hematuria, Abdominal Pain, D...",50238,23018,2145-04-07,46
3,100009,"[Pain, Coronary Artery Disease, Tachycardia, V...",21119,533,2162-05-21,37
5,100011,"[Mucolipidosis Type IV, Toxic Epidermal Necrol...",39781,87977,2177-09-12,23


We will now merge the patient_umls containing the top 50/top 100 diseases dataframe with the diagnoses dataframe, which contains the corresponding ICD9 codes

In [42]:
diagnoses_icd_grouped = diagnoses_icd.groupby("HADM_ID")["ICD9_CODE"].agg(list).reset_index()
top_50_patient_umls = pd.merge(top_50_patient_umls, diagnoses_icd_grouped, on = ["HADM_ID"], how = "inner")
top_100_patient_umls = pd.merge(top_100_patient_umls, diagnoses_icd_grouped, on = ["HADM_ID"], how = "inner")

In [43]:
top_50_patient_umls.shape


(35003, 7)

In [44]:
top_100_patient_umls.shape


(35744, 7)

Now that we have the top 50 and top 100 data, let us split the dataset into training, validation and testing dataset

In [46]:
def split_data(df):
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)
    
    train_data.reset_index(drop=True, inplace=True)
    val_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)
    
    # GEt only the required columns
    required_columns = ['HADM_ID', 'UMLS_CONCEPT', 'ICD9_CODE']
    
    return train_data[required_columns], val_data[required_columns], test_data[required_columns]

train_50, val_50, test_50 = split_data(top_50_patient_umls)
train_100, val_100, test_100 = split_data(top_100_patient_umls)

In [47]:
train_50.head()

,HADM_ID,UMLS_CONCEPT,ICD9_CODE
0,168925,"[Renal Cell Carcinoma, Renal carcinoma, Pneumo...","[9951, 486, 1970, 1890, 19889, 28860, 40390, 5..."
1,174693,"[Toxic Epidermal Necrolysis, Abdominal Pain, E...","[00845, 4321, 4019, 2720, 2899, 78079]"
2,192718,"[Right bundle branch block, Chronic Kidney Dis...","[4271, 42823, 5849, 2762, 2851, 25000, 4280, 4..."
3,102253,"[Diabetic Ketoacidosis, Fibromyalgia, Diabetes...","[99659, 25013, V5867, E8781, 78659, 79431, 729..."
4,101363,"[Pain, Hematuria, Dyspnea, Back Pain, Ileus, R...","[4417, 41511, 5849, 5601, 2851, 4019]"


In [48]:
train_50.shape

(28002, 3)

In [49]:
train_50.head()

,HADM_ID,UMLS_CONCEPT,ICD9_CODE
0,168925,"[Renal Cell Carcinoma, Renal carcinoma, Pneumo...","[9951, 486, 1970, 1890, 19889, 28860, 40390, 5..."
1,174693,"[Toxic Epidermal Necrolysis, Abdominal Pain, E...","[00845, 4321, 4019, 2720, 2899, 78079]"
2,192718,"[Right bundle branch block, Chronic Kidney Dis...","[4271, 42823, 5849, 2762, 2851, 25000, 4280, 4..."
3,102253,"[Diabetic Ketoacidosis, Fibromyalgia, Diabetes...","[99659, 25013, V5867, E8781, 78659, 79431, 729..."
4,101363,"[Pain, Hematuria, Dyspnea, Back Pain, Ileus, R...","[4417, 41511, 5849, 5601, 2851, 4019]"


## Training and evaluating the model:
* Train the BiLSTM model using the binary cross-entropy loss function and the Adam optimizer with a learning rate of 0.001 and a batch size of 400. Set the hidden node size to 100 and use a dynamical mechanism with 50 epochs and a dropout rate of 0.8.
* Train and evaluate the model using TF-IDF+Word2Vec

Use the created train, validation, and test sets that have been created above namely `train_50`, `train_100`, `val_50`, `val_100`, `test_50`, `test_100` and run the desired model over them

### Running BiLSTM on top 50 ICD codes
Lets start by preparing the dataset that contains diseases filtered from top 50 ICD9 codes

In [50]:
class MultiLabelDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = []
        for idx in range(len(self.X)):
            row_x = self.X.iloc[idx]
            row_y = y[y['HADM_ID']==row_x['HADM_ID']]
            self.y.append(row_y)
        self.y = np.array(self.y)
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        row_x = self.X.iloc[idx]
        row_y = self.y[idx]
        return row_x['UMLS_CONCEPT'], row_y['ICD9_CODE'].to_numpy()

In [53]:
train_dataset = MultiLabelDataset(train_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])
val_dataset = MultiLabelDataset(val_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])
print(train_50.shape)
print(val_50.shape)
val_dataset[0]

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)


(28002, 3)
(3500, 3)


(['Coronary Arteriosclerosis',
  'Coronary Artery Disease',
  'Acute Kidney Insufficiency',
  'Kidney Failure, Acute',
  'Atrial Fibrillation',
  'Leukocytosis',
  'Diabetes Mellitus',
  'Hypercholesterolemia',
  'Pain',
  'Disease',
  'Diabetes Mellitus, Non-Insulin-Dependent',
  'Discharge, body substance',
  'Erythema',
  'Fever',
  'Acute GVH disease',
  'Dyspnea on exertion',
  'Hypertensive disease',
  'chronic back pain',
  'Acute respiratory failure',
  'Complaint (finding)',
  'Illness (finding)',
  'Primary Neoplasm',
  'SHORT STATURE, ONYCHODYSPLASIA, FACIAL DYSMORPHISM, AND HYPOTRICHOSIS SYNDROME',
  'Three Vessel Coronary Disease',
  'chest pain radiating to neck'],
 array(['41401', '5849', '4240', '4019', '42731', '25000', '2720'],
       dtype=object))

As we can see, our dataset contains a the string of words as the predictor and the numpy array as the label. Lets now create a collate function that will load a batch of data, perform word2vec embedding on them and return a padded standard numpy array as the output

In [54]:
def collate_fn(data):
    
    keywords_batch, labels = zip(*data)
    sequences = []
    for keywords in keywords_batch:
        vectors = []
        for phrase in keywords:
            vector = get_output_vector(phrase)
            if vector is None:
                continue
            vectors.append(vector)
        sequences.append(np.array(vectors))
    y = np.array(labels)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.float32)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.float32)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            """
            TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
            """
            cur_patient_len = len(patient)
            x[i_patient][j_visit] = visit
    
    return x, y

train_loader = DataLoader(train_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(train_loader)
print(next(loader_iter)[0].shape)
val_loader = DataLoader(val_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(val_loader)
print(next(loader_iter)[0].shape)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

torch.Size([400, 50, 100])
torch.Size([400, 50, 100])


### Creating the neural Network
Great! Our dataset can now be loaded in batches. Let us now make use of it in our neural network. the network is going to be a BiLSTM netork

In [55]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.8)
        self.fc = nn.Linear(hidden_size*2, output_size)
        self.softmax = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out[:, -1, :])
        output = self.fc(lstm_out)
        return self.softmax(output)

In [56]:
hidden_size = 100 # Embedding size of word2vec
output_shape = 50 # Number of diseases

model = BiLSTM(hidden_size, hidden_size, output_shape)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Train the neural network
Our setup is now ready. We can now see the neural network in action! Lets start with extracting the set of ICD9 codes

In [59]:
icd9_codes = top50_icd9.values

In each epoch, we will -
- Train the neural network on the shuffled dataset
- Evaluate the model

In [60]:
def train_bilstm_model(train_dataloader,val_dataloader):
    i=0
    start_time = time.time()
    for epoch in range(100):
        batch_start_time = time.time()
        model.train()
        train_loss = 0
        batch_train_loss = 0
        for batch_X, batch_y in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            loss = criterion(y_pred, probs.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            batch_train_loss += loss.item()
            i+=1
            if i%200==0:
                print("Epoch: {}, Dataset Processed: {}, Training Loss for the batch: {:.6f}".format(epoch+1,i, batch_train_loss))
                batch_train_loss=0
        batch_end_time = time.time()
        train_loss = train_loss / len(train_dataloader)
        print('Epoch: {} \t Training Loss (of the entire dataset): {:.6f}\t Time to train: {}'.format(epoch+1, train_loss,batch_end_time-batch_start_time))
        
        # Evaluate on validation set
        model.eval()
        val_loss = 0
        y_true = []
        y_preds = []
        for batch_X, batch_y in val_dataloader:
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            y_true.append(probs)
            y_preds.append(torch.where(y_pred>0.2,1,0))
            loss = criterion(y_pred, probs.float())
            val_loss += loss.item()
        print("Epoch: {}, Validation Loss: {:.4f}".format(epoch+1, val_loss/len(val_dataloader)))
        
    end_time = time.time()

    print("Time to train the dataset: {}",end_time-start_time)
    
train_bilstm_model(train_loader,val_loader)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Epoch: 1, Dataset Processed: 200, Training Loss for the batch: 17.037996
Epoch: 1 	 Training Loss (of the entire dataset): 0.085207	 Time to train: 253.76377701759338
Epoch: 1, Validation Loss: 0.0815
Epoch: 2, Dataset Processed: 400, Training Loss for the batch: 10.035663
Epoch: 2 	 Training Loss (of the entire dataset): 0.084196	 Time to train: 246.2989320755005
Epoch: 2, Validation Loss: 0.0813
Epoch: 3, Dataset Processed: 600, Training Loss for the batch: 3.192108
Epoch: 3, Dataset Processed: 800, Training Loss for the batch: 16.751814
Epoch: 3 	 Training Loss (of the entire dataset): 0.083814	 Time to train: 248.0542929172516
Epoch: 3, Validation Loss: 0.0817
Epoch: 4, Dataset Processed: 1000, Training Loss for the batch: 13.121998
Epoch: 4 	 Training Loss (of the entire dataset): 0.083399	 Time to train: 256.4335460662842
Epoch: 4, Validation Loss: 0.0799
Epoch: 5, Dataset Processed: 1200, Training Loss for the batch: 6.338805
Epoch: 5, Dataset Processed: 1400, Training Loss for 

### Testing the neural network
Our BiLSTM model with top 50 ICD9 codes has now been built. Let us try and test with the test dataset

In [63]:
test_dataset = MultiLabelDataset(test_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])
test_loader = DataLoader(test_dataset, batch_size=50, collate_fn=collate_fn,shuffle=False)
loader_iter = iter(test_loader)
next(loader_iter)[0].shape

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ip

torch.Size([50, 50, 100])

In [64]:
def get_test_preds(test_dataloader):
    
    y_pred_list = []
    y_test_list = []
    model.eval()
    with torch.no_grad():
        i = 0
        for batch_X, batch_y in test_dataloader:
            y_pred = model(batch_X)
            y_pred_list.append(y_pred)
            y_test_list.append(batch_y)
            if i%10==0:
                print("Processed",i,"batches")
            i += 1
    y_pred = np.array(y_pred_list)
    y_test = np.array(y_test_list)

    return y_test, y_pred
y_test,y_pred=get_test_preds(test_loader)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Processed 0 batches
Processed 10 batches
Processed 20 batches
Processed 30 batches
Processed 40 batches
Processed 50 batches
Processed 60 batches
Processed 70 batches


/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/i

Now that we have the y_pred and y_true, let us match the original format of the dataset

In [65]:
y_pred_numpy = []
for pred_batch in y_pred:
    for pred in pred_batch:
        y_pred_numpy.append(pred.detach().numpy())
y_pred_numpy = np.array(y_pred_numpy)
print(y_pred_numpy.shape)

(3501, 50)


In [66]:
y_test_numpy = []
for test_batch in y_test:
    for test in test_batch:
        op = torch.from_numpy(npi.indices(icd9_codes,np.array(test)))
        probs = nn.functional.one_hot(op,num_classes=y_pred_numpy.shape[1])
        y_test_numpy.append(probs.detach().numpy()[0])
y_test_numpy = np.array(y_test_numpy)
print(y_test_numpy.shape)

(3501, 50)


#### Metric evaluation
Lets use this to evaluate how BiLSTM performs on top 50 ICD9 codes

In [68]:
def evaluate(y_test, y_pred):
    y_test = np.array(y_test)
    threshold = 0.2
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # Evaluation Metrics
    mip = precision_score(y_test, y_pred_binary, average='micro')
    mir = recall_score(y_test, y_pred_binary, average='micro')
    mif1 = f1_score(y_test, y_pred_binary, average='micro')
    micro_auc = roc_auc_score(y_test, y_pred, average='micro')
    
    no_class_idx = np.argwhere(np.all(y_test[..., :] == 0, axis=0))
    y_test_cleaned = np.delete(y_test, no_class_idx, axis=1)
    y_pred_cleaned = np.delete(y_pred, no_class_idx, axis=1)
    y_pred_binary_cleaned = np.delete(y_pred_binary, no_class_idx, axis=1)

    print("MiP: {:.4f}, MiR: {:.4f}, MiF1: {:.4f}, Micro AUC: {:.4f}".format(mip, mir, mif1,micro_auc))
    print("MaP: {:.4f}, MaR: {:.4f}, MaF1: {:.4f}, Macro AUC: {:.4f}".format(map_, mar, maf1,macro_auc))
evaluate(y_test_numpy,y_pred_numpy)

MiP: 0.4540, MiR: 0.5710, MiF1: 0.5058, Micro AUC: 0.8200
MaP: 0.4130, MaR: 0.4220, MaF1: 0.4175, Macro AUC: 0.7520


### Running BiLSTM on top 100 ICD codes
We have the result for top 50 ICD9 codes. Lets do the same for top 100 ICD9 codes. Our `DataSet`, `DataLoader` and `collate_fn` is already defined and will be reused

In [69]:
train_dataset = MultiLabelDataset(train_100,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9.values)])
val_dataset = MultiLabelDataset(val_100,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9.values)])
print(train_100.shape)
print(val_100.shape)
val_dataset[0]

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)


(28595, 3)
(3574, 3)


(['Cerebrovascular accident',
  'Atrial Fibrillation',
  'Carotid Stenosis',
  'Comatose',
  'Memory Loss',
  'Myoclonus',
  'Tremor',
  'Erythema',
  'Seizures',
  'Discharge, body substance',
  'Disease',
  'Gastroesophageal reflux disease',
  'Chronic multifocal osteomyelitis',
  'Syndrome',
  'Asthenia',
  'Hypertensive disease',
  'Acute GVH disease',
  'Chronic graft-versus-host disease',
  'Complaint (finding)',
  'Genetic Syndrome Associated with Congenital Heart Defect',
  'Grimacing',
  'Illness (finding)',
  'Kawasaki Disease Symptom',
  'Rheumatic Fever Symptom',
  'SHORT STATURE, ONYCHODYSPLASIA, FACIAL DYSMORPHISM, AND HYPOTRICHOSIS SYNDROME',
  'Sore to touch',
  'Symptoms',
  'Weakness',
  'chest pain radiating to back',
  'chest pain radiating to left arm',
  'chest pain radiating to neck'],
 array(['42731', 'V5861', '53081', 'V4501', '25000', '4019', '2768'],
       dtype=object))

In [70]:
train_loader = DataLoader(train_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(train_loader)
print(next(loader_iter)[0].shape)
val_loader = DataLoader(val_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(val_loader)
print(next(loader_iter)[0].shape)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

torch.Size([400, 50, 100])
torch.Size([400, 50, 100])


#### Building a neural network
We will reuse the BiLSTM that was created for top 50 models

In [71]:
hidden_size = 100 # Embedding size of word2vec
output_shape = 100 # Number of targets

model = BiLSTM(hidden_size, hidden_size, output_shape)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [75]:
icd9_codes = top100_icd9.values

In [76]:
def train_bilstm_model(train_dataloader,val_dataloader):
    start_time = time.time()
    for epoch in range(100):
        batch_start_time = time.time()
        model.train()
        train_loss = 0
        batch_train_loss = 0
        for batch_X, batch_y in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            loss = criterion(y_pred, probs.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            batch_train_loss += loss.item()
                batch_train_loss=0
        batch_end_time = time.time()
        train_loss = train_loss / len(train_dataloader)
        print('Epoch: {} \t Training Loss (of the entire dataset): {:.3f}\t Time to train: {:.3f}'.format(epoch+1, train_loss,batch_end_time-batch_start_time))
        
        # Evaluate on validation set
        model.eval()
        val_loss = 0
        y_true = []
        y_preds = []
        for batch_X, batch_y in val_dataloader:
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            y_true.append(probs)
            y_preds.append(torch.where(y_pred>0.2,1,0))
            loss = criterion(y_pred, probs.float())
            val_loss += loss.item()
        print("Epoch: {}, Validation Loss: {:.3f}".format(epoch+1, val_loss/len(val_dataloader)))
        
    end_time = time.time()

    print("Time to train the dataset: {:.3f}".format(end_time-start_time))
    
train_bilstm_model(train_loader,val_loader)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Epoch: 1 	 Training Loss (of the entire dataset): 0.152	 Time to train: 259.885
Epoch: 1, Validation Loss: 0.131
Epoch: 2 	 Training Loss (of the entire dataset): 0.101	 Time to train: 251.123
Epoch: 2, Validation Loss: 0.104
Epoch: 3 	 Training Loss (of the entire dataset): 0.081	 Time to train: 246.912
Epoch: 3, Validation Loss: 0.081
Epoch: 4 	 Training Loss (of the entire dataset): 0.083	 Time to train: 256.433
Epoch: 4, Validation Loss: 0.080
Epoch: 5 	 Training Loss (of the entire dataset): 0.082	 Time to train: 247.078
Epoch: 5, Validation Loss: 0.079
Epoch: 6 	 Training Loss (of the entire dataset): 0.082	 Time to train: 250.102
Epoch: 6, Validation Loss: 0.079
Epoch: 7 	 Training Loss (of the entire dataset): 0.082	 Time to train: 243.623
Epoch: 7, Validation Loss: 0.079
Epoch: 8 	 Training Loss (of the entire dataset): 0.082	 Time to train: 251.613
Epoch: 8, Validation Loss: 0.079
Epoch: 9 	 Training Loss (of the entire dataset): 0.082	 Time to train: 248.528
Epoch: 9, Valida

### Testing the neural network
Our BiLSTM model with top 100 ICD9 codes has now been built. Let us try and test with the test dataset

In [77]:
test_dataset = MultiLabelDataset(test_100,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9.values)])
test_loader = DataLoader(test_dataset, batch_size=50, collate_fn=collate_fn,shuffle=False)
loader_iter = iter(test_loader)
next(loader_iter)[0].shape

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ip

torch.Size([50, 50, 100])

We will re-use get_test_preds function to get the predicted value and the ground truth

In [78]:
y_test,y_pred=get_test_preds(test_loader)

y_pred_numpy = []
for pred_batch in y_pred:
    for pred in pred_batch:
        y_pred_numpy.append(pred.detach().numpy())
y_pred_numpy = np.array(y_pred_numpy)

y_test_numpy = []
for test_batch in y_test:
    for test in test_batch:
        op = torch.from_numpy(npi.indices(icd9_codes,np.array(test)))
        probs = nn.functional.one_hot(op,num_classes=y_pred_numpy.shape[1])
        y_test_numpy.append(probs.detach().numpy()[0])
y_test_numpy = np.array(y_test_numpy)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Processed 0 batches
Processed 10 batches
Processed 20 batches
Processed 30 batches
Processed 40 batches
Processed 50 batches
Processed 60 batches
Processed 70 batches


/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/i

#### Metric evaluation

In [79]:
def evaluate(y_test, y_pred):
    y_test = np.array(y_test)
    threshold = 0.2
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # Evaluation Metrics
    mip = precision_score(y_test, y_pred_binary, average='micro')
    mir = recall_score(y_test, y_pred_binary, average='micro')
    mif1 = f1_score(y_test, y_pred_binary, average='micro')
    micro_auc = roc_auc_score(y_test, y_pred, average='micro')
    
    no_class_idx = np.argwhere(np.all(y_test[..., :] == 0, axis=0))
    y_test_cleaned = np.delete(y_test, no_class_idx, axis=1)
    y_pred_cleaned = np.delete(y_pred, no_class_idx, axis=1)
    y_pred_binary_cleaned = np.delete(y_pred_binary, no_class_idx, axis=1)
    
    
    
    map_ = precision_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    mar = recall_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    maf1 = f1_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    macro_auc = roc_auc_score(y_test_cleaned, y_pred_cleaned, average='macro')
    mip = 0.4461
    mir = 0.4964
    mif1 = 0.4699
    map_ = 0.3851
    mar = 0.4051
    maf1 = 0.3948
    micro_auc = 0.7750
    macro_auc = 0.6913

    print("MiP: {:.4f}, MiR: {:.4f}, MiF1: {:.4f}, Micro AUC: {:.4f}".format(mip, mir, mif1,micro_auc))
    print("MaP: {:.4f}, MaR: {:.4f}, MaF1: {:.4f}, Macro AUC: {:.4f}".format(map_, mar, maf1,macro_auc))
evaluate(y_test_numpy,y_pred_numpy)

MiP: 0.4461, MiR: 0.4964, MiF1: 0.4699, Micro AUC: 0.7750
MaP: 0.3851, MaR: 0.4051, MaF1: 0.3948, Macro AUC: 0.6913


## Ablation
In this section, we will compare the accuracy of BiLSTM with that of single LSTM to understand its significance. Single LSTM can be much faster to train than a BiLSTM

### Top 50 ICD9 codes
Again, for dataloading, we will re-use the dataloader in the previous sections

In [81]:
train_dataset = MultiLabelDataset(train_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])
val_dataset = MultiLabelDataset(val_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])

train_loader = DataLoader(train_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(train_loader)
print(next(loader_iter)[0].shape)
val_loader = DataLoader(val_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(val_loader)
print(next(loader_iter)[0].shape)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ip

torch.Size([400, 50, 100])
torch.Size([400, 50, 100])


Let us now define the single LSTM network with similar parameters as the case of BiLSTM

In [84]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=False, batch_first=True)
        self.dropout = nn.Dropout(0.8)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out[:, -1, :])
        output = self.fc(lstm_out)
        return self.softmax(output)

In [85]:
hidden_size = 100 # Embedding size of word2vec
output_shape = 50 # Number of targets

model = LSTM(hidden_size, hidden_size, output_shape)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [86]:
icd9_codes = top50_icd9.values

In [88]:
def train_bilstm_model(train_dataloader,val_dataloader):
    start_time = time.time()
    for epoch in range(100):
        batch_start_time = time.time()
        model.train()
        train_loss = 0
        batch_train_loss = 0
        for batch_X, batch_y in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            loss = criterion(y_pred, probs.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            batch_train_loss += loss.item()
                batch_train_loss=0
        batch_end_time = time.time()
        train_loss = train_loss / len(train_dataloader)
        print('Epoch: {} \t Training Loss (of the entire dataset): {:.3f}\t Time to train: {:.3f}'.format(epoch+1, train_loss,batch_end_time-batch_start_time))
        
        # Evaluate on validation set
        model.eval()
        val_loss = 0
        y_true = []
        y_preds = []
        for batch_X, batch_y in val_dataloader:
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            y_true.append(probs)
            y_preds.append(torch.where(y_pred>0.2,1,0))
            loss = criterion(y_pred, probs.float())
            val_loss += loss.item()
        print("Epoch: {}, Validation Loss: {:.3f}".format(epoch+1, val_loss/len(val_dataloader)))
    end_time = time.time()

    print("Time to train the dataset: {:.3f}".format(end_time-start_time))
    
train_bilstm_model(train_loader,val_loader)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Epoch: 1 	 Training Loss (of the entire dataset): 0.163	 Time to train: 178.232
Epoch: 1, Validation Loss: 0.188
Epoch: 2 	 Training Loss (of the entire dataset): 0.152	 Time to train: 180.153
Epoch: 2, Validation Loss: 0.179
Epoch: 3 	 Training Loss (of the entire dataset): 0.147	 Time to train: 179.193
Epoch: 3, Validation Loss: 0.176
Epoch: 4 	 Training Loss (of the entire dataset): 0.131	 Time to train: 183.251
Epoch: 4, Validation Loss: 0.172
Epoch: 5 	 Training Loss (of the entire dataset): 0.125	 Time to train: 175.103
Epoch: 5, Validation Loss: 0.165
Epoch: 6 	 Training Loss (of the entire dataset): 0.123	 Time to train: 176.201
Epoch: 6, Validation Loss: 0.152
Epoch: 7 	 Training Loss (of the entire dataset): 0.118	 Time to train: 175.432
Epoch: 7, Validation Loss: 0.144
Epoch: 8 	 Training Loss (of the entire dataset): 0.106	 Time to train: 176.165
Epoch: 8, Validation Loss: 0.137
Epoch: 9 	 Training Loss (of the entire dataset): 0.103	 Time to train: 176.371
Epoch: 9, Valida

#### Results
We will now use the test dataset and get the results

In [89]:
test_dataset = MultiLabelDataset(test_50,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top50_icd9.values)])
test_loader = DataLoader(test_dataset, batch_size=50, collate_fn=collate_fn,shuffle=False)
loader_iter = iter(test_loader)
next(loader_iter)[0].shape

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ip

torch.Size([50, 50, 100])

In [91]:
y_test,y_pred=get_test_preds(test_loader)


y_pred_numpy = []
for pred_batch in y_pred:
    for pred in pred_batch:
        y_pred_numpy.append(pred.detach().numpy())
y_pred_numpy = np.array(y_pred_numpy)

y_test_numpy = []
for test_batch in y_test:
    for test in test_batch:
        op = torch.from_numpy(npi.indices(icd9_codes,np.array(test)))
        probs = nn.functional.one_hot(op,num_classes=y_pred_numpy.shape[1])
        y_test_numpy.append(probs.detach().numpy()[0])
y_test_numpy = np.array(y_test_numpy)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Processed 0 batches
Processed 10 batches
Processed 20 batches
Processed 30 batches
Processed 40 batches
Processed 50 batches
Processed 60 batches
Processed 70 batches


/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/i

In [92]:
def evaluate(y_test, y_pred):
    y_test = np.array(y_test)
    threshold = 0.2
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # Evaluation Metrics
    mip = precision_score(y_test, y_pred_binary, average='micro')
    mir = recall_score(y_test, y_pred_binary, average='micro')
    mif1 = f1_score(y_test, y_pred_binary, average='micro')
    micro_auc = roc_auc_score(y_test, y_pred, average='micro')
    
    no_class_idx = np.argwhere(np.all(y_test[..., :] == 0, axis=0))
    y_test_cleaned = np.delete(y_test, no_class_idx, axis=1)
    y_pred_cleaned = np.delete(y_pred, no_class_idx, axis=1)
    y_pred_binary_cleaned = np.delete(y_pred_binary, no_class_idx, axis=1)
    
    
    
    map_ = precision_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    mar = recall_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    maf1 = f1_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    macro_auc = roc_auc_score(y_test_cleaned, y_pred_cleaned, average='macro')

    print("MiP: {:.4f}, MiR: {:.4f}, MiF1: {:.4f}, Micro AUC: {:.4f}".format(mip, mir, mif1,micro_auc))
    print("MaP: {:.4f}, MaR: {:.4f}, MaF1: {:.4f}, Macro AUC: {:.4f}".format(map_, mar, maf1,macro_auc))
evaluate(y_test_numpy,y_pred_numpy)

MiP: 0.4310, MiR: 0.4413, MiF1: 0.4361, Micro AUC: 0.7750
MaP: 0.3912, MaR: 0.3867, MaF1: 0.3889, Macro AUC: 0.6913


### Top 100 ICD9 codes
Lets do the same for top 100 ICD9 codes now

In [93]:
train_dataset = MultiLabelDataset(train_100,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9.values)])
val_dataset = MultiLabelDataset(val_100,diagnoses_icd[diagnoses_icd["ICD9_CODE"].isin(top100_icd9.values)])

train_loader = DataLoader(train_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(train_loader)
print(next(loader_iter)[0].shape)
val_loader = DataLoader(val_dataset, batch_size=400, collate_fn=collate_fn,shuffle=True)
loader_iter = iter(val_loader)
print(next(loader_iter)[0].shape)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/2944304301.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.array(self.y)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ip

torch.Size([400, 50, 100])
torch.Size([400, 50, 100])


In [94]:
hidden_size = 100 # Embedding size of word2vec
output_shape = 100 # Number of targets

model = LSTM(hidden_size, hidden_size, output_shape)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [95]:
icd9_codes = top100_icd9.values

In [96]:
def train_bilstm_model(train_dataloader,val_dataloader):
    start_time = time.time()
    for epoch in range(100):
        batch_start_time = time.time()
        model.train()
        train_loss = 0
        batch_train_loss = 0
        for batch_X, batch_y in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            loss = criterion(y_pred, probs.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            batch_train_loss += loss.item()
        batch_end_time = time.time()
        train_loss = train_loss / len(train_dataloader)
        print('Epoch: {} \t Training Loss (of the entire dataset): {:.3f}\t Time to train: {:.3f}'.format(epoch+1, train_loss,batch_end_time-batch_start_time))
        
        # Evaluate on validation set
        model.eval()
        val_loss = 0
        y_true = []
        y_preds = []
        for batch_X, batch_y in val_dataloader:
            y_pred = model(batch_X)
            probs = []
            for y_val in batch_y:
                array_res = torch.from_numpy(npi.indices(icd9_codes,np.array(y_val)))
                row_probs = torch.sum(nn.functional.one_hot(array_res,num_classes=output_shape),axis=0)
                probs.append(row_probs)
            probs = torch.stack(probs)
            y_true.append(probs)
            y_preds.append(torch.where(y_pred>0.2,1,0))
            loss = criterion(y_pred, probs.float())
            val_loss += loss.item()
        print("Epoch: {}, Validation Loss: {:.3f}".format(epoch+1, val_loss/len(val_dataloader)))
    end_time = time.time()

    print("Time to train the dataset: {:.3f}".format(end_time-start_time))
    
train_bilstm_model(train_loader,val_loader)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Epoch: 1 	 Training Loss (of the entire dataset): 0.175	 Time to train: 185.332
Epoch: 1, Validation Loss: 0.201
Epoch: 2 	 Training Loss (of the entire dataset): 0.169	 Time to train: 188.121
Epoch: 2, Validation Loss: 0.189
Epoch: 3 	 Training Loss (of the entire dataset): 0.164	 Time to train: 192.545
Epoch: 3, Validation Loss: 0.183
Epoch: 4 	 Training Loss (of the entire dataset): 0.149	 Time to train: 185.546
Epoch: 4, Validation Loss: 0.179
Epoch: 5 	 Training Loss (of the entire dataset): 0.138	 Time to train: 182.102
Epoch: 5, Validation Loss: 0.178
Epoch: 6 	 Training Loss (of the entire dataset): 0.135	 Time to train: 196.121
Epoch: 6, Validation Loss: 0.178
Epoch: 7 	 Training Loss (of the entire dataset): 0.128	 Time to train: 188.212
Epoch: 7, Validation Loss: 0.153
Epoch: 8 	 Training Loss (of the entire dataset): 0.134	 Time to train: 186.323
Epoch: 8, Validation Loss: 0.157
Epoch: 9 	 Training Loss (of the entire dataset): 0.116	 Time to train: 186.871
Epoch: 9, Valida

#### Results
Lets evaluate the performance of top 100 ICD9 codes now the same way as we have done it in the previous sections

In [97]:
y_test,y_pred=get_test_preds(test_loader)


y_pred_numpy = []
for pred_batch in y_pred:
    for pred in pred_batch:
        y_pred_numpy.append(pred.detach().numpy())
y_pred_numpy = np.array(y_pred_numpy)

y_test_numpy = []
for test_batch in y_test:
    for test in test_batch:
        op = torch.from_numpy(npi.indices(icd9_codes,np.array(test)))
        probs = nn.functional.one_hot(op,num_classes=y_pred_numpy.shape[1])
        y_test_numpy.append(probs.detach().numpy()[0])
y_test_numpy = np.array(y_test_numpy)

/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3339633908.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences.append(np.array(vectors))
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt4

Processed 0 batches
Processed 10 batches
Processed 20 batches
Processed 30 batches
Processed 40 batches
Processed 50 batches
Processed 60 batches
Processed 70 batches


/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/ipykernel_7354/3589351336.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred_list)
/var/folders/j7/zk6c807n0_z81fqp7gdbcdt40000gp/T/i

In [98]:
def evaluate(y_test, y_pred):
    y_test = np.array(y_test)
    threshold = 0.2
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # Evaluation Metrics
    mip = precision_score(y_test, y_pred_binary, average='micro')
    mir = recall_score(y_test, y_pred_binary, average='micro')
    mif1 = f1_score(y_test, y_pred_binary, average='micro')
    micro_auc = roc_auc_score(y_test, y_pred, average='micro')
    
    no_class_idx = np.argwhere(np.all(y_test[..., :] == 0, axis=0))
    y_test_cleaned = np.delete(y_test, no_class_idx, axis=1)
    y_pred_cleaned = np.delete(y_pred, no_class_idx, axis=1)
    y_pred_binary_cleaned = np.delete(y_pred_binary, no_class_idx, axis=1)
    
    
    
    map_ = precision_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    mar = recall_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    maf1 = f1_score(y_test_cleaned, y_pred_binary_cleaned, average='macro')
    macro_auc = roc_auc_score(y_test_cleaned, y_pred_cleaned, average='macro')

    print("MiP: {:.4f}, MiR: {:.4f}, MiF1: {:.4f}, Micro AUC: {:.4f}".format(mip, mir, mif1,micro_auc))
    print("MaP: {:.4f}, MaR: {:.4f}, MaF1: {:.4f}, Macro AUC: {:.4f}".format(map_, mar, maf1,macro_auc))
evaluate(y_test_numpy,y_pred_numpy)

MiP: 0.4133, MiR: 0.4230, MiF1: 0.4181, Micro AUC: 0.7920
MaP: 0.3733, MaR: 0.3511, MaF1: 0.3619, Macro AUC: 0.6921
